In [4]:
# Clear any logs from previous runs

In [1]:
import numpy as np
import pandas as pd
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split

In [3]:
np.random.seed(0)

tf.config.optimizer.set_jit(True)
scaler = StandardScaler()

### Conjunto de datos: SDSS DR17

In [4]:
url = "https://raw.githubusercontent.com/igomezv/neurapprox/main/SDSS/star_classification.csv"
data = pd.read_csv(url)
cols = ['alpha','delta','u','g','r','i','z','redshift','class']
data = data[cols]
data.head()

,alpha,delta,u,g,r,i,z,redshift,class
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,0.634794,GALAXY
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,0.779136,GALAXY
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,0.644195,GALAXY
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,0.932346,GALAXY
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,0.116123,GALAXY


In [5]:
data["class"]=[0 if i == "GALAXY" else 1 if i == "STAR" else 2 for i in data["class"]]
print(data.head())
data = data.to_numpy()

        alpha      delta         u         g         r         i         z  \
0  135.689107  32.494632  23.87882  22.27530  20.39501  19.16573  18.79371   
1  144.826101  31.274185  24.77759  22.83188  22.58444  21.16812  21.61427   
2  142.188790  35.582444  25.26307  22.66389  20.60976  19.34857  18.94827   
3  338.741038  -0.402828  22.13682  23.77656  21.61162  20.50454  19.25010   
4  345.282593  21.183866  19.43718  17.58028  16.49747  15.97711  15.54461   

   redshift  class  
0  0.634794      0  
1  0.779136      0  
2  0.644195      0  
3  0.932346      0  
4  0.116123      0  


In [6]:
# Divide data into X and Y and implement hot_ones in Y
def prepare_dataset(data):
    X, Y = np.empty((0)), np.empty((0))
    X = data[:,0:8]
    Y = data[:,8]
    Y = to_categorical(Y, num_classes=3)
    return X, Y

In [7]:
# Split dataset into train, validation and test sets
X,Y = prepare_dataset(data)

# Defines ratios, w.r.t. whole dataset.
ratio_train = 0.8
ratio_val = 0.1
ratio_test = 0.1

# Produces test split.
x_, X_test, y_, Y_test = split(X, Y, test_size = ratio_test, random_state=0)

# Adjusts val ratio, w.r.t. remaining dataset.
ratio_remaining = 1 - ratio_test
ratio_val_adjusted = ratio_val / ratio_remaining

# Produces train and val splits.
X_train, X_val, Y_train, Y_val = split(x_, y_, test_size=ratio_val_adjusted, random_state=0)

# Normalize and scale the input sets.
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)
X_val   = scaler.transform(X_val)

lenx, input_shape = np.shape(X)

### Hiperparámetros del modelo

In [8]:
HP_LAYERS =    hp.HParam('layers', hp.Discrete([2, 3, 4]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 100, 200]))
HP_LEARNING  = hp.HParam('learning_rate', hp.Discrete([1e-5,1e-4,1e-3]))
# HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([64,128,256,512]))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', mode='max',
                                   min_delta=0,
                                   patience=6,
                                   restore_best_weights=True)]
batch_size = 128
epochs = 50

In [9]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_LAYERS,
                 HP_NUM_UNITS,
                 HP_LEARNING],
        metrics=[hp.Metric('loss', display_name="Accuracy")])

In [10]:
def train_test_model(hparams):    
    
    # Train LSTM model and predict on validation set
    model = tf.keras.Sequential()
    model.add(Input(shape=(int(X_train.shape[1]),)))
    model.add(Dense(hparams[HP_NUM_UNITS], input_shape=(int(X_train.shape[1]),)))
    
    for i in range(hparams[HP_LAYERS]):        
        model.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(Dense(3, activation=tf.nn.softmax))
     
    optimizer = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss=CategoricalCrossentropy(),
            metrics=["categorical_accuracy"])
    
    # Run with 1 epoch to speed things up for demo purposes

    model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=batch_size, shuffle=True, verbose=0)

    _, loss = model.evaluate(X_test, Y_test)
    
    return loss

In [11]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)
        return loss

In [12]:
session_num = 0
datos = []

for deep_layers in HP_LAYERS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for learning_rate in HP_LEARNING.domain.values:
#             for batch_size in HP_BATCHSIZE.domain.values:
            t = time.time()
            hparams = {

                HP_LAYERS: deep_layers,
                HP_NUM_UNITS: num_units,
                HP_LEARNING: learning_rate,
#                     HP_BATCHSIZE: batch_size
            }
            run_name = "run-%d" % session_num
            print('\n--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            score = run('logs/hparam_tuning/' + run_name, hparams)
            t = time.time()-t
            session_num += 1
            print("Accuracy:", score, "Tiempo transcurrido:", t)
            
            datos.append([deep_layers, num_units, learning_rate, score, t])

print(session_num)


--- Starting trial: run-0
{'layers': 2, 'num_units': 50, 'learning_rate': 1e-05}
313/313 [==============================] - 0s 590us/step - loss: 0.1930 - categorical_accuracy: 0.9450
Accuracy: 0.9449999928474426 Tiempo transcurrido: 24.957477807998657

--- Starting trial: run-1
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001}
313/313 [==============================] - 0s 605us/step - loss: 0.1101 - categorical_accuracy: 0.9689
Accuracy: 0.9689000248908997 Tiempo transcurrido: 20.00023913383484

--- Starting trial: run-2
{'layers': 2, 'num_units': 50, 'learning_rate': 0.001}
313/313 [==============================] - 0s 584us/step - loss: 0.0995 - categorical_accuracy: 0.9714
Accuracy: 0.9714000225067139 Tiempo transcurrido: 15.150592803955078

--- Starting trial: run-3
{'layers': 2, 'num_units': 100, 'learning_rate': 1e-05}
313/313 [==============================] - 0s 613us/step - loss: 0.1519 - categorical_accuracy: 0.9577
Accuracy: 0.9577000141143799 Tiempo transcurrido: 29

### Guardar datos

In [13]:
filename = "historial_sdss_tunning.txt"
df = pd.DataFrame(datos, columns = ["Deep size", "Num units", "Learning rate", "Accuracy", "Tiempo de ejecución"])

df.sort_values(by=["Accuracy", "Tiempo de ejecución"], ascending=[0,0], ignore_index=True, inplace=True)

df.to_csv(filename, header=True, index=False, sep='\t', mode='w') # a=append, w=overwrite

In [14]:
df

,Deep size,Num units,Learning rate,Accuracy,Tiempo de ejecución
0,2,200,0.00100,0.9729,23.103655
1,4,100,0.00100,0.9728,20.806302
2,4,200,0.00100,0.9726,38.246722
3,3,100,0.00100,0.9722,15.220246
4,2,100,0.00100,0.9721,25.116137
5,2,50,0.00100,0.9714,15.150593
6,3,200,0.00100,0.9713,18.099978
7,3,100,0.00010,0.9712,33.204466
8,4,200,0.00010,0.9709,23.549818
9,4,50,0.00100,0.9706,11.812055


In [15]:
np.sum(df[["Tiempo de ejecución"]])/60

Tiempo de ejecución    11.281182
dtype: float64

In [16]:
# rm -rf /tmp/tb_logs/

### Now in terminal:
`python3 -m tensorboard.main --logdir='/home/isidro/Documents/github/neurapprox/logs/hparam_tuning'`